## MySQL
***


### Instalação

Rode o docker do mysql

```
pip3 install mysql-connector
```

### Entrar no MySQL

```
mysql -u <username> -p
senha: ****
exit
```

Se precisar colocar o host como localhost use o -h

### Backup

```
mysqldump -u <username> -p <database> > backup.sql
```


***
### Conexão Root
***

In [1]:
from mysql import connector

In [2]:
host = "mysql"

In [3]:
# Connection é o objeto que representa o banco de dados
conexao = connector.connect(
    host=host,
    user="root",
    passwd="notebook"
)

In [4]:
# Vamos criar um cursor, ou seja, um iterador que permite navegar e manipular os registros do DB.
cursor = conexao.cursor()

***
### Criação de um usuário e inserir permissões
***

Lista de privilegios que pode ser garantido a um usuário: [Privileges](https://dev.mysql.com/doc/refman/8.0/en/privileges-provided.html#privileges-provided-summary)

In [5]:
# Crie o usuário
username = "fulano"
password = "django1234"
cursor.execute(f"CREATE USER '{username}'@'{host}' IDENTIFIED BY '{password}'")

***
### Criar banco de dados
***

In [6]:
# Cria um database e use ela
db = "mydb"
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db}")
cursor.execute(f"USE {db};")

In [7]:
# Mostrar os banco de dados
cursor.execute("SHOW DATABASES")

In [8]:
for database in cursor:
    print(database)

('information_schema',)
('mydb',)
('mysql',)
('performance_schema',)


In [9]:
# Garanta previlegios e permita que ele conceda essas mesmas permissões a outros usuários (WITH GRANT OPTION)
cursor.execute(f"GRANT SELECT, INSERT, UPDATE, DELETE, CREATE, ALTER, DROP, REFERENCES ON mydb.* TO '{username}'@'{host}';")
# Recarregar as tabelas de concessão para garantir que os novos privilégios sejam aplicados
cursor.execute("FLUSH PRIVILEGES")

***
### Criando uma tabela
***

In [10]:
# Criar uma tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS cidades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    cidade VARCHAR(255),
    uf VARCHAR(2)
);
""")

In [11]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nome VARCHAR(255) NOT NULL,
    cpf VARCHAR(11) NOT NULL,
    email VARCHAR(100) NOT NULL,
    cidade_id INT,
    criado_em DATE NOT NULL,
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);
""")

In [12]:
# Mostrar tabelas
cursor.execute("""SHOW TABLES""")

In [13]:
for table in cursor:
    print(table)

('cidades',)
('clientes',)


***
### Inserindo registros
***

In [14]:
cidades = [('Campinas','SP'),
           ('Sao Paulo','SP'),
           ('Rio de Janeiro','RJ')]

In [15]:
cursor.executemany("""
INSERT INTO cidades (cidade, uf) VALUES (%s, %s)
""", cidades)

In [16]:
clientes = [('Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08'),
            ('Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09'),
            ('Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09'),
            ('Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')]

In [17]:
cursor.executemany("""
INSERT INTO clientes (nome, cpf, email, cidade_id, criado_em) VALUES (%s, %s, %s, %s, %s)
""", clientes)

In [18]:
# Salvando os dados no BD.
conexao.commit()

In [19]:
# Verifica quantos inserções foram feitas
print(cursor.rowcount, "dados inseridos.")
# Pega o ID do ultimo elemento inseridp
print("ID:", cursor.lastrowid)

4 dados inseridos.
ID: 1


***
### Lendo os dados
***

In [20]:
# Registros de clientes
cursor.execute("""
SELECT * FROM clientes;
""")

In [21]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


In [22]:
# Registros de cidades
cursor.execute("""
SELECT * FROM cidades;
""")

In [23]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Campinas', 'SP')
(2, 'Sao Paulo', 'SP')
(3, 'Rio de Janeiro', 'RJ')


In [24]:
# Registros de clientes e cidades
cursor.execute("""
SELECT * FROM clientes INNER JOIN cidades ON clientes.cidade_id = cidades.id;
""")

In [25]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8), 1, 'Campinas', 'SP')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8), 3, 'Rio de Janeiro', 'RJ')


***
### Alterando os dados
***

In [26]:
id_cliente = 1
novo_email = 'regis_novo@gmail.com'
novo_criado_em = '2014-06-11'

In [27]:
cursor.execute("""
UPDATE clientes
SET email = %s, criado_em = %s
WHERE id = %s
""", (novo_email, novo_criado_em, id_cliente))

In [28]:
conexao.commit()

In [29]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [30]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


***
### Deletando um registro
***

In [31]:
id_cliente = 4

In [32]:
cursor.execute("""
DELETE FROM clientes
WHERE id = %s
""", (id_cliente,))

In [33]:
conexao.commit()

In [34]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [35]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))


In [36]:
# Deleta todas as instâncias
cursor.execute("DELETE FROM clientes")
cursor.execute("DELETE FROM cidades")

In [37]:
conexao.commit()

***
### Adicionar uma nova coluna
***

In [38]:
cursor.execute("""
ALTER TABLE clientes
ADD COLUMN bloqueado BOOLEAN;
""")

In [39]:
conexao.commit()

***
### Fechando conexão
***

In [40]:
#cursor.execute(f"REVOKE ALL PRIVILEGES ON mydb.* FROM '{username}'@'{host}'")
cursor.execute("DROP TABLE IF EXISTS clientes")
cursor.execute("DROP TABLE IF EXISTS cidades")
cursor.execute(f"DROP USER '{username}'@'{host}'")
cursor.execute(f"DROP DATABASE IF EXISTS {db}")
conexao.commit()

In [41]:
cursor.close()
conexao.close()